In [1]:
import pandas as pd 

In [2]:
import wandb
api = wandb.Api()
runs = api.runs("wuzhengx/fine_tuning")
eval_metrics = {
    "sst3":"eval/Macro-F1",
    "mrpc":"eval/accuracy",
    "qnli":"eval/accuracy",
}

In [3]:
task_name = "qnli" # please change to the task you are analyzing.

In [4]:
all_data = []
for run in runs:
    if task_name in run.name:
        perf_metrics = run.history(keys=[eval_metrics[task_name]])[eval_metrics[task_name]]
        best_perf_metrics = max(perf_metrics)
        run_name = run.name
        
        name_list = run_name.split("_")
        for i in range(len(name_list)):
            if name_list[i] == "seed":
                seed = int(name_list[i+1])
            if name_list[i] == "reverse":
                if name_list[i+1] == "True":
                    reverse_order = True
                else:
                    reverse_order = False
            if name_list[i] == "random":
                if name_list[i+1].strip("/") == "True":
                    random_order = True
                else:
                    random_order = False
            if name_list[i] == "data":
                if len(name_list[i+1].split("-")) > 2:
                    perturbed_type = "-".join(name_list[i+1].split("-")[2:])
                else:
                    perturbed_type = "null"
            if name_list[i] == "inoculation":
                inoculation_p = float(name_list[i+1])
        all_data.append([perturbed_type, reverse_order, random_order, inoculation_p, seed, best_perf_metrics])

In [5]:
df = pd.DataFrame(
    all_data, 
    columns = ['perturbed_type','reverse_order','random_order', 'inoculation_p', 'seed', 'best_perf_metrics']
)

In [6]:
df[(df["perturbed_type"]=="null")&(df["random_order"]==False)&(df["reverse_order"]==False)].sort_values("inoculation_p")

,perturbed_type,reverse_order,random_order,inoculation_p,seed,best_perf_metrics
2,null,False,False,0.0,42,0.931539
1,null,False,False,1.0,42,0.931539


In [7]:
df[(df["perturbed_type"]=="null")&df["random_order"]].sort_values("inoculation_p")

,perturbed_type,reverse_order,random_order,inoculation_p,seed,best_perf_metrics
6,null,False,True,0.0,42,0.881018
5,null,False,True,1.0,42,0.885228


In [8]:
df[(df["perturbed_type"]=="null")&df["reverse_order"]].sort_values("inoculation_p")

,perturbed_type,reverse_order,random_order,inoculation_p,seed,best_perf_metrics
4,null,True,False,0.0,42,0.896760
3,null,True,False,1.0,42,0.908292


In [9]:
df[(df["perturbed_type"]=="en~fr@N~fr@V")].sort_values("inoculation_p")

,perturbed_type,reverse_order,random_order,inoculation_p,seed,best_perf_metrics
0,en~fr@N~fr@V,False,False,0.0,42,0.907560
10,en~fr@N~fr@V,False,False,1.0,42,0.914882


In [10]:
df[(df["perturbed_type"]=="en~jaktc@N~jaktc@V")].sort_values("inoculation_p")

,perturbed_type,reverse_order,random_order,inoculation_p,seed,best_perf_metrics
11,en~jaktc@N~jaktc@V,False,False,0.0,42,0.912868
9,en~jaktc@N~jaktc@V,False,False,1.0,42,0.918360


In [11]:
df[(df["perturbed_type"]=="en~fr@N~jaktc@V")].sort_values("inoculation_p")

,perturbed_type,reverse_order,random_order,inoculation_p,seed,best_perf_metrics
8,en~fr@N~jaktc@V,False,False,0.0,42,0.904448
7,en~fr@N~jaktc@V,False,False,1.0,42,0.913784
